## Importing

In [2]:
import pandas as pd 
import numpy as np
import plotly.express as px
import datetime as dt
import pycountry

C:\Users\User\AppData\Roaming\Python\Python37\site-packages\matplotlib\docstring.py:92: MatplotlibDeprecationWarning: The dedent function was deprecated in Matplotlib 3.1 and will be removed two minor releases later. Use inspect.cleandoc instead.
  func.__doc__ = func.__doc__ and cbook.dedent(func.__doc__)
C:\Users\User\AppData\Roaming\Python\Python37\site-packages\matplotlib\patches.py:2405: MatplotlibDeprecationWarning: The dedent function was deprecated in Matplotlib 3.1 and will be removed two minor releases later. Use inspect.cleandoc instead.
  __doc__ = cbook.dedent(__doc__) % \
C:\Users\User\AppData\Roaming\Python\Python37\site-packages\matplotlib\patches.py:3086: MatplotlibDeprecationWarning: The dedent function was deprecated in Matplotlib 3.1 and will be removed two minor releases later. Use inspect.cleandoc instead.
  __doc__ = cbook.dedent(__doc__) % \
C:\Users\User\AppData\Roaming\Python\Python37\site-packages\matplotlib\patches.py:3891: MatplotlibDeprecationWarning: The 

## Reading Data

In [3]:
data = pd.read_csv(".\Data\\aviation_accidents in countries - aviation_accidents.csv")

In [4]:
num_accidents = data.shape[0]

In [5]:
data.head()

,Country,date,Air-craft type,registration name/mark,operator,fatilites,location,category
0,Australia,07-Dec-2016,Boeing 737-8FE (WL),VH-VUP,Virgin Australia Airlines,0,Hobart Airpo...,A2
1,Ukraine,19-Feb-1987,Antonov An-26,NaN,Soviet AF,9,near Stadnitsa,A1
2,Iran,19-Feb-2003,Ilyushin Il-76MD,15-2280,Iranian Revolutionary Guard,275,near Kerman,A1
3,Germany,11-May-1990,Transall C-160D,50+39,German AF,10,near Rodenbach,A1
4,Russia,04-Jan-1943,Junkers Ju-52/3m,NaN,German AF,0,Ostrow,A1


## Questions to answer:
1. Countries where the most accidents happend. [done]
2. Opertaors that had the most accidents. [done]
2.b. operator by country and country by operator[done ]
3. Air-craft type that had the most accidents. [done Air-craft by accident cause and Air-craft by accident result]
4. Count of Accidents per decade in each country. [done]
5. Count of Accidents per category. [done]
6. Count of total loss accidents out of all accidents. (If an airplane had an accidents, how propably it's going to be a total loss?). [done]
7. Count of Accidents per category for each Air-craft type. (If **** type made an accidents how probably it's gonna be a total loss?). [done]
8. Count of accidents per operator in each country. [done]
9. Count of accidents per country in each operator. [done]


## Discovering the data

In [6]:
# How many null values do we have?
nulls_count_precentage={ col_name : (data[col_name].isna().sum(), 
                                str( round( 100 * data[col_name].isna().sum() / data.shape[0] ,2))+" %") 
                                for col_name in data.columns}
nulls_count_precentage

{'Country': (0, '0.0 %'),
 'date': (68, '0.31 %'),
 'Air-craft type': (68, '0.31 %'),
 'registration name/mark': (1453, '6.68 %'),
 'operator': (70, '0.32 %'),
 'fatilites': (3067, '14.11 %'),
 'location': (419, '1.93 %'),
 'category': (71, '0.33 %')}

In [7]:
# How many unique values do we have?
data.nunique()

Country                     227
date                      14244
Air-craft type             3056
registration name/mark    19855
operator                   5697
fatilites                   359
location                  13401
category                     11
dtype: int64

## Studying the data regarding the countries

The dataset contains a column of 'Country' values, for further visualizing, we will need the code of the country. The following function does that task.

In [8]:
def get_country_code(country):
    """
    This function returns the ISO Code of a country.
    
    Argument:
    country -- A string value holds a country name
    
    Returns:
    result -- The ISO Alpha-3 code of 'country' argument.
    """
    try:
        result = pycountry.countries.search_fuzzy(country)
    except Exception:
        return np.nan
    else:
        return result[0].alpha_3
iso_map = {country: get_country_code(country) for country in data["Country"].unique()}

In [9]:
data["country_code"] = data["Country"].map(iso_map)

### Finding the number of accidents for each country

In [10]:
countries = data.groupby(['Country','country_code']).size().sort_values(ascending=[False])
countries_df=countries.to_frame().reset_index()
countries_df = countries_df.rename(columns= {0: 'count_accidents'})
countries_df[:15]

,Country,country_code,count_accidents
0,USA,USA,4303
1,Russia,RUS,1410
2,United Kingdom,GBR,836
3,Canada,CAN,816
4,India,IND,696
5,Brazil,BRA,606
6,France,FRA,546
7,Germany,DEU,509
8,China,CHN,507
9,Colombia,COL,455


#### Accidents per country - visulization 1 pie chart 

In [11]:
fig = px.pie(countries_df[:15], 
            values='count_accidents', 
            names='Country', 
            title='15 countries where the most accidents happend')
fig.update_traces( textinfo='value+label',textfont_size=10)
fig.show()

In [12]:
def precentage(integer):
    return integer*100/num_accidents
output_list = list(map(precentage, countries_df['count_accidents'][:15]))
  
print(output_list)

[19.791187563241653, 6.485143960997148, 3.845092447796891, 3.7531045901940945, 3.201177444577316, 2.7872320853647317, 2.5112685125563425, 2.341090975991169, 2.3318921902308896, 2.0927237604636186, 1.9685401526998436, 1.6189862938092172, 1.5821911507680986, 1.572992365007819, 1.4718057216447429]


#### Accidents per country - visulization 2 Map

In [13]:
fig = px.scatter_geo(countries_df, locations="country_code",
                     color="Country", # which column to use to set the color of markers
                     hover_name="Country", # column added to hover information
                     size="count_accidents", # size of markers
                     projection="natural earth")
fig.show()

In [14]:
# Keeping the top 20 countries where the most accidents happened
countries_most_accidents = countries_df['Country'][:20]

## Operator

### Fixing operator values



Keep operator name only before the comma, some operators names are joined with city name. Ex: 'Aeroflot, Yakutsk' 

In [15]:
data["operator"]=data["operator"].str.split(',').str[0]

In [16]:
operators = data.groupby(['operator']).size().sort_values(ascending=[False]).to_frame().reset_index().rename(columns= {0: 'count_accidents'})
fig = px.pie(operators[:10], 
            values='count_accidents', 
            names='operator', 
            title='10 operators which suffered the most number of accidents')
fig.update_traces( textinfo='value+label',textfont_size=10)
fig.show()

In [17]:

# Keeping the top 20 operators that had the most accidents
operators_most_accidents = operators['operator'][:20]
# operators that had more than 50 accidents
operators_50 = operators[operators['count_accidents'] > 50] 

## Country and operator

For the sake of more clear visualization we will focus on the top 20 operators that had more the most accidents in the top 20 countries where the most accidents happened

In [19]:
countries_operators = data.groupby(['Country','country_code','operator']).size().sort_values(ascending=[False])
countries_operators_df=countries_operators.to_frame().reset_index()
countries_operators_df = countries_operators_df.rename(columns= {0: 'count_accidents'})
countries_operators_df[:2]

,Country,country_code,operator,count_accidents
0,Russia,RUS,Aeroflot,849
1,USA,USA,USAF,441


### Drawing the accidents per each operator in each country

In [191]:
fig = px.treemap(countries_operators_df, path=[px.Constant('world'), 'Country', 'operator'], values='count_accidents',
                   hover_data=['country_code'])
fig.update_traces(root_color="lightgrey")
fig.show()

In [20]:
countries_operators_df= countries_operators_df[countries_operators_df['Country'].isin(countries_most_accidents)]
countries_operators_df= countries_operators_df[countries_operators_df['operator'].isin(operators_most_accidents)]

### In each country (of the top 20), what operators (of the top 20) made the most accidents


In [180]:
countries_operators_50_df= countries_operators_df[countries_operators_df['count_accidents']>50]

In [21]:
fig = px.treemap(countries_operators_df, path=[px.Constant('world'), 'Country', 'operator'], values='count_accidents',
                   hover_data=['country_code'])
fig.update_traces(root_color="lightgrey")
fig.show()

### In each operator (of the top 20),  in which countries (of the top 20) those operators made the most accidents


In [22]:
fig = px.treemap(countries_operators_df, path=[px.Constant('world'), 'operator','Country'], values='count_accidents',
                   hover_data=['country_code'])
fig.update_traces(root_color="lightgrey")
fig.show()

## Studying Regarding the category of the accident

Accident Categories:
- A = Accident
- I = Incident
- H = Hijacking
- C = Criminal occurrence (sabotage, shoot down)
- O = other occurrence (ground fire, sabotage)
- U = type of occurrence unknown

- 1 = hull-loss
- 2 = repairable damage

### Fixing the category representation in the data

In [60]:
categories_filtered=data.dropna(subset=['category'])

In [7]:
def cause_accident(value):
    #value=(value).str
    if value.find('A')>=0:
        return 'Accident'
    if value.find('I')>=0:
        return 'Incident'
    if value.find('H')>=0:
        return 'Hijacking'
    if value.find('C')>=0:
        return 'Criminal occurrence'
    if value.find('O')>=0:
        return 'Other occurrence'
    if value.find('U')>=0:
        return 'Unknown'

def result_accident(value):
    #value=str(value)
    if value.find('1')>=0:
        return 'Hull Loss'
    if value.find('2')>=0:
        return 'Repairable Damage'

In [61]:
causes = categories_filtered['category'].map(cause_accident)
results = categories_filtered['category'].map(result_accident)
categories_filtered = pd.concat([categories_filtered, causes, results], axis=1, join="inner")
categories_filtered.columns = ['Country', 'date', 'Air-craft type', 'registration name/mark',
       'operator', 'fatilites', 'location', 'category', 'country_code',
       'Accident_cause', 'Accident_result']
categories_filtered.head()

,Country,date,Air-craft type,registration name/mark,operator,fatilites,location,category,country_code,Accident_cause,Accident_result
0,Australia,07-Dec-2016,Boeing,VH-VUP,Virgin Australia Airlines,0,Hobart Airpo...,A2,AUS,Accident,Repairable Damage
1,Ukraine,19-Feb-1987,Antonov,NaN,Soviet AF,9,near Stadnitsa,A1,UKR,Accident,Hull Loss
2,Iran,19-Feb-2003,Ilyushin,15-2280,Iranian Revolutionary Guard,275,near Kerman,A1,IRN,Accident,Hull Loss
3,Germany,11-May-1990,Transall,50+39,German AF,10,near Rodenbach,A1,DEU,Accident,Hull Loss
4,Russia,04-Jan-1943,Junkers,NaN,German AF,0,Ostrow,A1,RUS,Accident,Hull Loss


### Causes

In [62]:
result_cause = categories_filtered.groupby(
                                ['Accident_cause','Accident_result']).size().sort_values(ascending=False).to_frame().reset_index().rename(columns={0: 'count_accidents'})

In [144]:
fig = px.histogram(result_cause, x="Accident_cause", y="count_accidents", color="Accident_cause")
fig.show()

### Results

In [63]:
fig = px.histogram(result_cause, x="Accident_result", y="count_accidents", color="Accident_result")
fig.show()

### accident results cause vs accident accident result

In [64]:
hull_loss_df = result_cause[result_cause['Accident_result']=='Hull Loss']
repairable_df = result_cause[result_cause['Accident_result']=='Repairable Damage']
result_by_cause_df = pd.merge(hull_loss_df, repairable_df, how='outer', on = 'Accident_cause').fillna(0)

In [65]:
to_draw= [result_by_cause_df["count_accidents_x"].to_list(),result_by_cause_df["count_accidents_y"].to_list()]

In [74]:
def precentage(integer):
    return [round(i*100/num_accidents,2) for i in integer]
output_list = list(map(precentage, to_draw))
print(output_list)

[[72.73, 5.04, 4.32, 1.48, 0.19, 0.0], [10.62, 0.36, 0.29, 0.0, 4.59, 0.05]]


In [73]:
fig = px.imshow(to_draw,
                labels=dict(x="Cause of Accident", y="Result of Accident", color="Accidents"),
                x= result_by_cause_df["Accident_cause"],
                y=["Hull Loss","Repairable Damage"],
                text_auto = True
               )
fig.update_xaxes(side="top" )
#fig.update_traces( text='value+label',textfont_size=10)
fig.show()

In [ ]:
fig = px.sunburst(result_cause, path=['Accident_cause', 'Accident_result'], values='count_accidents',
                  color='count_accidents')
fig.show()

In [ ]:
fig = px.sunburst(result_cause, path=['Accident_result','Accident_cause'], values='count_accidents',
                  color='count_accidents')
fig.show()

In [ ]:
fig = px.bar(result_cause, x="Accident_cause", y="count_accidents", color="Accident_result", 
 pattern_shape="Accident_result", pattern_shape_sequence=[".", "x"])
             
fig.show()

### Number of full loss accidents out of all accidents + operator

In [141]:
result_operator= categories_filtered.groupby(
                                ['operator','Accident_result']).size().sort_values(ascending=False).to_frame().reset_index().rename(columns={0: 'count_accidents'})
result_operator_50= result_operator[result_operator['count_accidents']>70]
fig = px.sunburst(result_operator_50, path=['Accident_result', 'operator'], values='count_accidents',
                  color='count_accidents')
fig.show()                                

### Number of cause of accidents out of all accidents + operator

In [142]:
cause_operator= categories_filtered.groupby(
                                ['operator','Accident_cause']).size().sort_values(ascending=False).to_frame().reset_index().rename(columns={0: 'count_accidents'})
cause_operator_50=cause_operator[cause_operator['count_accidents']>70]
fig = px.sunburst(cause_operator_50, path=['Accident_cause', 'operator'], values='count_accidents',
                  color='count_accidents')
fig.show()    

### Countries, cause of accident, result of accident

In [215]:
country_category=categories_filtered.groupby(
                      ['Country', 'Accident_result','Accident_cause']
                      ).size().sort_values(ascending=False).to_frame(
                      ).reset_index().rename(columns={0: 'count_accidents'})

In [220]:
country_category_50=country_category[country_category['count_accidents']>100]

In [221]:
fig = px.bar(country_category_50, x="Accident_cause", y="count_accidents", color="Country", 
 pattern_shape="Accident_result", pattern_shape_sequence=[".", "x"])
             
fig.show()

## Studying Regarding the Air Craft type

### Fixing Air Craft type Colomn

In [21]:
data["Air-craft type"]=data["Air-craft type"].str.split(' ').str[0]
categories_filtered["Air-craft type"]=categories_filtered["Air-craft type"].str.split(' ').str[0]

### Which Aircraft has the most accidents ?

In [38]:
air_crafts = data.groupby(['Air-craft type']).size().sort_values(ascending=[False]).to_frame().reset_index().rename(columns= {0: 'count_accidents'})
fig = px.pie(air_crafts[:10], 
            values='count_accidents', 
            names='Air-craft type', 
            title='10 air_crafts which suffered the most number of accidents')
fig.update_traces( textinfo='value+label',textfont_size=10)
fig.show()

In [53]:
most_4_air_crafts=air_crafts['Air-craft type'][:4]

### Number of full loss accidents out of all accidents + Air Craft

In [23]:
result_air_craft_type= categories_filtered.groupby(
                                ['Air-craft type','Accident_result']).size().sort_values(ascending=False).to_frame().reset_index().rename(columns={0: 'count_accidents'})

In [24]:
result_air_craft_type_50= result_air_craft_type[result_air_craft_type['count_accidents']>100]

In [25]:
fig = px.sunburst(result_air_craft_type_50, path=['Accident_result', 'Air-craft type'], values='count_accidents',
                  color='count_accidents')
fig.show()

### Number of accidents per cause of accident + Air craft

In [26]:
cause_air_craft_type= categories_filtered.groupby(
                                ['Air-craft type','Accident_cause']).size().sort_values(ascending=False).to_frame().reset_index().rename(columns={0: 'count_accidents'})

In [27]:
cause_air_craft_type_50= cause_air_craft_type[cause_air_craft_type['count_accidents']>100]

In [28]:
fig = px.sunburst(cause_air_craft_type_50, path=['Accident_cause', 'Air-craft type'], values='count_accidents',
                  color='count_accidents')
fig.show()

In [54]:
cause_air_craft_type_filtered= cause_air_craft_type[cause_air_craft_type['Air-craft type'].isin(most_4_air_crafts)]

In [55]:
fig = px.sunburst(cause_air_craft_type_filtered, path=['Accident_cause', 'Air-craft type'], values='count_accidents',
                  color='count_accidents')
fig.show()

In [33]:
air_craft_category=categories_filtered.groupby(
                      ['Air-craft type', 'Accident_result','Accident_cause']
                      ).size().sort_values(ascending=False).to_frame(
                      ).reset_index().rename(columns={0: 'count_accidents'})

In [36]:
air_craft_category

,Air-craft type,Accident_result,Accident_cause,count_accidents
0,Douglas,Hull Loss,Accident,4249
1,Antonov,Hull Loss,Accident,1354
2,Lockheed,Hull Loss,Accident,941
3,Curtiss,Hull Loss,Accident,784
4,Boeing,Hull Loss,Accident,637
...,...,...,...,...
853,DHC-8-106,Hull Loss,Accident,1
854,DHC-8-103B,Repairable Damage,Accident,1
855,DHC-8-101,Repairable Damage,Accident,1
856,DHC-7-110,Repairable Damage,Other occurrence,1


In [56]:
air_craft_category_filtered= air_craft_category[air_craft_category['Air-craft type'].isin(most_4_air_crafts)]

In [49]:
fig = px.bar(air_craft_category_filtered, x="Air-craft type", y="count_accidents", color="Accident_cause", 
 pattern_shape="Accident_result", pattern_shape_sequence=[".", "x"])
fig.show()

## Editing the date value
The date is given in day-month-year format, some of the values are missing, they are either Null values or explicitly written 'Date unk'. Besides, some of the values contians question marks where a part of the date is missing. 

We will create a new colomn contains the year data, still we'll have a lot of years. We will add another colomn that contains the decade of when the accident happened.

#### Dropping rows with unknow dates

In [193]:
dates_filtered = data.drop(data[(data.date == 'date unk.') ].index)

#### Dropping rows with null dates

In [195]:
dates_filtered=dates_filtered.dropna(subset=['date'])

#### Adding a year & decade colomns

In [197]:
# Year
dates_filtered['year'] = dates_filtered['date'].str[-4:].str.lower()

In [198]:
dates_filtered.nunique()

Country                     160
date                      14243
Air-craft type             3052
registration name/mark    19824
operator                   5279
fatilites                   359
location                  13385
category                     11
country_code                151
year                        104
dtype: int64

In [200]:
# Decade
dates_filtered['decade'] = dates_filtered['year'].astype(int).round(-1)
dates_filtered['decade'] 

0        2020
1        1990
2        2000
3        1990
4        1940
         ... 
21737    2020
21738    2020
21739    2020
21740    2020
21741    2020
Name: decade, Length: 21638, dtype: int32

#### Finding the number of accidents for each country in each decade

In [201]:
decade_country= dates_filtered.groupby(['Country','country_code','decade']).size().to_frame().reset_index().rename(columns={0: 'accidents'})

In [94]:
#decade_country=decade_country['decade'].sort_values(ascending=[False])
decade_country

,Country,country_code,decade,accidents
0,Afghanistan,AFG,1960,3
1,Afghanistan,AFG,1970,3
2,Afghanistan,AFG,1980,20
3,Afghanistan,AFG,1990,35
4,Afghanistan,AFG,2000,21
...,...,...,...,...
1236,Zimbabwe,ZWE,1980,8
1237,Zimbabwe,ZWE,1990,1
1238,Zimbabwe,ZWE,2000,5
1239,Zimbabwe,ZWE,2010,3


In [101]:
fig = px.line(decade_country, x="decade", y="accidents", color="Country", line_group="Country", hover_name="Country",
        line_shape="spline", render_mode="svg")
fig.show()

In [99]:
decade_country['decade']=decade_country.decade.sort_values()

In [100]:
fig = px.choropleth(decade_country, locations="country_code", color="accidents", hover_name="Country", animation_frame="decade", range_color=[1,600])
fig.show()

In [230]:
#decade_country_50.accidents.sort_values()[50]
decade_country_50= decade_country[decade_country['accidents']>50].sort_values(['decade'])

In [208]:
df = px.data.medals_long()

fig = px.bar(decade_country, x="decade", y="accidents", color="Country")
             
fig.show()

In [214]:
fig = px.bar(decade_country_50, x="decade", y="accidents", color="Country")
             
fig.show()

In [231]:
fig = px.line(decade_country_50, x="decade", y="accidents", color="Country", line_group="Country", hover_name="Country",
        line_shape="spline", render_mode="svg")
fig.show()